# NOTIFICATIONS MVP

### 0. libraries

In [67]:
import pandas as pd
import numpy as np
from tqdm import tqdm_notebook
import pydantic
import http
import jinja2 

### 1. database

In [59]:
def get_movies_db(n_movies):    
    L = []
    for _ in range(n_movies):
        d = {}
        d["id"] = _+1
        d["title"] = f"movie{_+1}"
        d["imdb_rating"] = np.around(np.random.uniform(0, 10), 1)
        L.append(d)
    df = pd.DataFrame.from_records(L)
    del L
    return df

def get_users_db(n_users):    
    L = []
    for _ in range(n_users):
        d = {}
        d["id"] = _+1
        d["full_name"] = f"full_name{_+1}"
        d["email"] = f"email{_+1}"
        L.append(d)
    df = pd.DataFrame.from_records(L)
    del L
    return df

def get_templates_db():  
    d = {"id":1, "template":"Dear {{ user_full_name }}, new {{ movie_title }} series has been released", "vars":"movie_title,user_full_name"}
    df = pd.DataFrame.from_records([d])
    return df

In [60]:
df_movies_db = get_movies_db(1000)
df_users_db = get_users_db(10000)
df_templates_db = get_templates_db()

### dto

In [61]:
class MovieDTO(pydantic.BaseModel):
    title: str
    
class UserDTO(pydantic.BaseModel):
    full_name: str
    email: str
    
class TemplateDTO(pydantic.BaseModel):
    template: str
    vars: str
    
class ResponseDTO(pydantic.BaseModel):
    data:  MovieDTO | UserDTO | TemplateDTO | None
    status: http.HTTPStatus

### service

In [62]:
class MovieService:
    def __init__(self):
        self.db = df_movies_db.copy()
        
    async def get_by_id(self, id:int, dto: MovieDTO) -> ResponseDTO:
        return ResponseDTO(
            data = dto(**self.db.query("id==@id").iloc[0].to_dict()),
            status = http.HTTPStatus.OK
        )
        
class UserService:
    def __init__(self):
        self.db = df_users_db.copy()
        
    async def get_by_id(self, id:int, dto: UserDTO) -> ResponseDTO:
        return ResponseDTO(
            data = dto(**self.db.query("id==@id").iloc[0].to_dict()),
            status = http.HTTPStatus.OK
        )
        
        
class TemplateService:
    def __init__(self):
        self.db = df_templates_db.copy()
        
    async def get_by_id(self, id:int, dto: TemplateDTO) -> ResponseDTO:
        return ResponseDTO(
            data = dto(**self.db.query("id==@id").iloc[0].to_dict()),
            status = http.HTTPStatus.OK
        )

In [63]:
movie_service = MovieService()
await movie_service.get_by_id(1, MovieDTO)

ResponseDTO(data=MovieDTO(title='movie1'), status=<HTTPStatus.OK: 200>)

In [64]:
user_service = UserService()
await user_service.get_by_id(1, UserDTO)

ResponseDTO(data=UserDTO(full_name='full_name1', email='email1'), status=<HTTPStatus.OK: 200>)

In [65]:
template_service = TemplateService()
await template_service.get_by_id(1, TemplateDTO)

ResponseDTO(data=TemplateDTO(template='Dear {{ user_full_name }}, new {{ movie_title }} series has been released', vars='movie_title,user_full_name'), status=<HTTPStatus.OK: 200>)

### api gateway
* microservices:
1. movies
2. users
3. templates

In [ ]:
class NotificationService:
    async def send_email(
        self,
        template_id: int,
        movie_id: int,
        users_id: tuple[int]
        
    ):   
        pass
    
    async def send_push(self):
        pass
    
    async def send_telegram(self):
        pass

In [48]:
# api gateway
def send_email_notification(
    template_id: int,
    movie_id: int,
    users_id: tuple[int]
):
    # ----- RabbitMQ -> RPC call templates service
    # ----- RabbitMQ <- data about user
    d_user = {...} 
    
    # ----- RabbitMQ -> RPC call movies service
    # ----- RabbitMQ <- data about movie
    d_movie = {...} 
    
    # ----- RabbitMQ -> RPC call users service
    # ----- RabbitMQ <- data about template
    d_template = {...} 
    
    jinja_template = d_template["template"]
    vars = d_template["vars"]
    rtemplate = jinja2.Environment(loader=jinja2.BaseLoader).from_string(jinja_template)
    message = rtemplate.render(**vars)
    
    # ----- RabbitMQ -> RPC call notification service(send email)
    send_email(message, user.email)
    # send_push(message, user.email)
    # send_telegram(message, user.email)


,id,template,vars
0,1,"Уважаемый {{ user_full_name }}, вышла новавя с...","movie_title,user_full_name"
